Header

In [ ]:
import numpy as np
import time
import healpy as hp
import matplotlib.pyplot as plt

import cupy as cp

import cunusht
from cunusht import get_geom

import delensalot
from delensalot import utils
from delensalot.sims.sims_lib import Xunl, Xsky, Xobs


lmax, mmax = 1024, 1024
ll = np.arange(0,lmax)
geominfo = ('gl',{'lmax': lmax})
# geominfo = ('healpix',{'nside': 512})

solver = 'cufinufft'
backend = 'GPU'

synunl = Xunl(lmax=lmax, geominfo=geominfo)
synsky = Xsky(lmax=lmax, unl_lib=synunl, geominfo=geominfo)

philm = synunl.get_sim_phi(0, space='alm')
dlm = hp.almxfl(philm, np.sqrt(ll*(ll+1)))

Tunl = synunl.get_sim_unl(0, spin=0, space='alm', field='temperature')
Tsky = synsky.get_sim_sky(0, spin=0, space='map', field='temperature')

solver = 'cufinufft'
backend = 'GPU'
mode = 'nuFFT'
t = cunusht.get_transformer(solver, mode, backend)
t.set_geometry(geominfo)
# int(t.geom.nph[0]/2-1)


if solver in ['finufft']:
    defres, ptg, map_dfs = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
    print("defres.shape: {}".format(defres.shape))
elif solver in ['duccnufft']:
    defres, ptg, map_dfs = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
    print("defres.shape: {}".format(defres.shape))
elif solver == 'ducc':
    defres = t.gclm2lenmap(Tunl.copy(), dlm=dlm, mmax=lmax, spin=0, backwards=False)
    print("defres.shape: {}".format(defres.shape))
elif solver == 'cufinufft':
    defres, ptg, map_dfs = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
    print("defres.shape: {}".format(defres.shape))

INFO:: 02-28 18:12:50:: delensalot.sims.sims_lib.__init__ - sht_solver not given, defaulting to ducc
INFO:: 02-28 18:12:50:: delensalot.sims.sims_lib.__init__ - sht_backend not given, defaulting to CPU
INFO:: 02-28 18:12:50:: delensalot.sims.sims_lib.__init__ - sht_solver not given, defaulting to ducc
INFO:: 02-28 18:12:50:: delensalot.sims.sims_lib.__init__ - sht_backend not given, defaulting to CPU
deflection std is 2.53e+00 amin
build_angles
defres.shape: (1026, 2160)


In [8]:
phis = t.geom.rings2phi(t.geom, np.arange(len(t.geom.theta)))
thetas = np.array([th*np.ones(shape=t.geom.nph[0]) for th in t.geom.theta]).flatten()
map_dfs_shifted = np.fft.fftshift(map_dfs, axes=(0,1))

# tskyducc = defres.reshape(lmax+1,-1)# np.roll(defres.reshape(lmax+1,-1),int(t.geom.nph[0]/2+1), axis=1)

tunlducc = np.roll(t.synthesis(Tunl.copy(), spin=0, lmax=lmax, mmax=mmax, nthreads=4)[0].reshape(lmax+3,-1),0, axis=1)
tskyducc = np.roll(defres.reshape(lmax+1,-1).copy(),int(t.geom.nph[0]/2-1), axis=1)

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(16,5))
ax[0].imshow(defres.reshape(lmax+1,-1), vmin=-500, vmax=500)
ax[1].imshow(tunlducc.reshape(lmax+1,-1), vmin=-500, vmax=500)
ax[2].imshow(defres.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1), vmin=-80, vmax=80, label='finufft')
ax[3].imshow(Tsky.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1), vmin=-80, vmax=80, label='ducc')


# hp.mollview(defres, norm='hist')
# plt.colorbar()

In [ ]:
import numpy as np
import finufft
a = []
ffacs = np.arange(1.0091,1.01,0.2)
for ffac in ffacs:#,1e-7,1e-10]:
    tol = 1e-13
    f = map_dfs_shifted
    # f = np.ones_like(mapd_dfs_shifted)
    tunl_ = finufft.nufft2d2(x=phis, y=thetas.copy(), f=f.T, eps=tol, isign=1)
    Tskyfi_ = finufft.nufft2d2(x=ptg[:,1][::-1].copy(), y=ptg[:,0].copy(), f=f.T, eps=tol, isign=1)
    # Tskyfi_ = finufft.nufft2d2(x=ptg[:,0][::-1].copy(), y=ptg[:,1].copy(), f=f, eps=tol)
# int(t.geom.nph[0]/2+
    tunlfi = np.roll(np.real(tunl_).reshape(lmax+1,-1),0, axis=1)
    tskyfi = np.roll(np.real(Tskyfi_).reshape(lmax+1,-1),0, axis=1)
        
    # tunlfi = np.real(tunl_).reshape(lmax+1,-1)
    # tskyfi = np.real(Tskyfi_).reshape(lmax+1,-1)
    
    # restpl, restpu = 10, 150
    # resttl, resttu = 10, 150
    restpl, restpu = 0, 2160
    resttl, resttu = 0, 1026

    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow((tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[0].set_title('ducc unlensed')
    ax[1].imshow((tunlfi)[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[1].set_title('fi unlensed')
    ax[2].imshow((tunlducc.reshape(lmax+1,-1)-tunlfi)[resttl:resttu,restpl:restpu], vmin=-1e-7,vmax=1e-7)
    ax[2].set_title('difference: exact zero')
    plt.show()


    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow(tskyducc.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[0].set_title('ducc lensed')
    ax[1].imshow(tskyfi[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[1].set_title('fi lensed')
    ax[2].imshow((tskyfi.reshape(lmax+1,-1)-tskyducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-10,vmax=10)
    ax[2].set_title('difference is at about 1e-7')
    plt.show()


    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-20,vmax=20)
    ax[0].set_title('ducc lensed - unlensed')
    ax[1].imshow((tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu][:,::-1]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-20,vmax=20)
    ax[1].set_title('fi lensed - unlensed')
    ax[2].imshow((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu]-(tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-1e-6,vmax=1e-6)
    plt.show()
    a.append(np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)[:,::-1]-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    print("{:.3f}".format(ffac), np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    

# isign -1

In [ ]:
"""
NEVER TOUCH THIS AGAIN..
"""

phis = t.geom.rings2phi(t.geom, np.arange(len(t.geom.theta)))
thetas = np.array([th*np.ones(shape=t.geom.nph[0]) for th in t.geom.theta]).flatten()
map_dfs_shifted = np.fft.fftshift(map_dfs, axes=(0,1))

tunlducc = np.roll(t.synthesis(Tunl.copy(), spin=0, lmax=lmax, mmax=mmax, nthreads=4)[0].reshape(lmax+1,-1),int(t.geom.nph[0]/2-1), axis=1)
tskyducc = np.roll(defres.reshape(lmax+1,-1).copy(), 0, axis=1)

a = []
ffacs = np.arange(1.0091,1.01,0.2)
for ffac in ffacs:#,1e-7,1e-10]:
    tol = 1e-13
    f = map_dfs_shifted

    tunl_ = finufft.nufft2d2(x=phis[::-1], y=thetas.copy(), f=f.T, eps=tol)
    Tskyfi_ = finufft.nufft2d2(x=ptg[:,1].copy(), y=ptg[:,0].copy(), f=f.T, eps=tol)

    # int(t.geom.nph[0]/2+
    tunlfi = np.roll(np.real(tunl_).reshape(lmax+1,-1),0, axis=1)
    tskyfi = np.roll(np.real(Tskyfi_).reshape(lmax+1,-1),0, axis=1)
    
    # restpl, restpu = 10, 150
    # resttl, resttu = 10, 150
    restpl, restpu = 0, 2160
    resttl, resttu = 0, 1026

    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow((tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[0].set_title('ducc unlensed')
    ax[1].imshow((tunlfi)[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[1].set_title('fi unlensed')
    ax[2].imshow((tunlducc.reshape(lmax+1,-1)-tunlfi)[resttl:resttu,restpl:restpu], vmin=-1e-8,vmax=1e-8)

    ax[2].set_title('difference: exact zero')
    plt.show()


    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow(tskyducc.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[0].set_title('ducc lensed')
    ax[1].imshow(tskyfi[resttl:resttu,restpl:restpu], vmin=-500,vmax=500)
    ax[1].set_title('fi lensed')
    ax[2].imshow(tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu]-tskyducc.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu],
                 vmin=-1,vmax=1)
    ax[2].set_title('difference is at about 1e-7')
    plt.show()


    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].imshow((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-20,vmax=20)
    ax[0].set_title('ducc lensed - unlensed')
    ax[1].imshow((tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu][:,::-1]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-20,vmax=20)
    ax[1].set_title('fi lensed - unlensed')
    ax[2].imshow((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu]-(tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-1e-6,vmax=1e-6)
    plt.show()
    a.append(np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)[:,::-1]-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    print("{:.3f}".format(ffac), np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    

## healpix grid

In [ ]:
import numpy as np
import time
import healpy as hp
import matplotlib.pyplot as plt

import cunusht
from cunusht import get_geom

import delensalot
from delensalot import utils
from delensalot.sims.sims_lib import Xunl, Xsky, Xobs

nside = 1024
lmax, mmax = 3*nside, 3*nside
ll = np.arange(0,lmax)
geominfo = ('healpix',{'nside': nside})
# geominfo = ('healpix',{'nside': 512})

solver = 'duccnufft'
backend = 'CPU'

synunl = Xunl(lmax=lmax, geominfo=geominfo)
synsky = Xsky(lmax=lmax, unl_lib=synunl, geominfo=geominfo)

philm = synunl.get_sim_phi(0, space='alm')
dlm = hp.almxfl(philm, np.sqrt(ll*(ll+1)))

Tunl = synunl.get_sim_unl(0, spin=0, space='alm', field='temperature')
Tsky = synsky.get_sim_sky(0, spin=0, space='map', field='temperature')


t = cunusht.get_transformer(solver, backend)
t.set_geometry(geominfo)

# tunlducc = t.synthesis(Tunl.copy(), spin=0, lmax=lmax, mmax=mmax, nthreads=4)[0].reshape(lmax+1,-1)
# tskyducc = np.roll(Tsky.reshape(lmax+1,-1).copy(),-1, axis=1)

tunlducc = t.synthesis(Tunl.copy(), spin=0, lmax=lmax, mmax=mmax, nthreads=4)[0]
tskyducc = Tsky.copy()

if solver in ['finufft']:
    defres, ptg, map_dfs = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
    print("defres.shape: {}".format(defres.shape))
elif solver in ['duccnufft']:
    defres, ptg, map_dfs = t.gclm2lenmap(Tunl.copy(), dlm=dlm, lmax=lmax, mmax=lmax, spin=0, nthreads=4)
    print("defres.shape: {}".format(defres.shape))
elif solver == 'ducc':
    defres = t.gclm2lenmap(Tunl.copy(), dlm=dlm, mmax=lmax, spin=0, backwards=False)
    print("defres.shape: {}".format(defres.shape))

In [ ]:
phis = t.geom.rings2phi(t.geom, np.arange(len(t.geom.theta)))
thetas = np.hstack([th*np.ones(shape=t.geom.nph[thi]) for thi,th in enumerate(t.geom.theta)])
print(phis.shape, thetas.shape)

map_dfs_shifted = np.fft.fftshift(map_dfs, axes=(0,1))
tskyducc = defres# np.roll(defres.reshape(lmax+1,-1),int(t.geom.nph[0]/2+1), axis=1)

In [ ]:
plt.plot(thetas[5000000:5010000])
plt.plot(phis[5000000:5010000])

healpix

In [ ]:
import numpy as np
import finufft
a = []
ffacs = np.arange(1.0091,1.01,0.2)
for ffac in ffacs:#,1e-7,1e-10]:
    tol = 1e-10
    f = map_dfs_shifted
    # f = np.ones_like(mapd_dfs_shifted)
    tunl_ = finufft.nufft2d2(x=phis, y=thetas.copy(), f=f.T, eps=tol, isign=1)
    Tskyfi_ = finufft.nufft2d2(x=ptg[:,1].copy(), y=ptg[:,0].copy(), f=f.T, eps=tol, isign=-1)
    # Tskyfi_ = finufft.nufft2d2(x=ptg[:,0][::-1].copy(), y=ptg[:,1].copy(), f=f, eps=tol)

    tunlfi = np.real(tunl_)
    tskyfi = np.real(Tskyfi_)
    # tunlfi = np.real(tunl_).reshape(lmax+1,-1)
    # tskyfi = np.real(Tskyfi_).reshape(lmax+1,-1)
    
    restpl, restpu = 400, 1200
    resttl, resttu = 1050, 1200
    restpl, restpu = 0, 2160
    resttl, resttu = 0, 1026

    hp.mollview(tunlducc)
    # hp.set_title('ducc unlensed')
    hp.mollview(tunlfi)
    # hp.set_title('fi unlensed')
    hp.mollview((tunlducc-tunlfi))
    # ax[2].set_title('difference: exact zero')
    plt.show()



    hp.mollview(tskyducc)
    # ax[0].set_title('ducc lensed')
    hp.mollview(tskyfi)
    # ax[1].set_title('fi lensed')
    hp.mollview(tskyfi-tskyducc, cbar=True)

    # ax[2].set_title('difference is at about 1e-7')
    plt.show()


    fig, ax = plt.subplots(1,3,figsize=(16,6))
    ax[0].mollview((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu], vmin=-20,vmax=20)
    ax[0].set_title('ducc lensed - unlensed')
    ax[1].mollview((tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu][:,::-1]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-20,vmax=20)
    ax[1].set_title('fi lensed - unlensed')
    ax[2].mollview((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1))[resttl:resttu,restpl:restpu]-(tskyfi.reshape(lmax+1,-1)[resttl:resttu,restpl:restpu]-tunlfi[resttl:resttu,restpl:restpu]), vmin=-1e-6,vmax=1e-6)
    plt.show()
    a.append(np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)[:,::-1]-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)[:,::-1]-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    print("{:.3f}".format(ffac), np.sqrt(np.sum(np.abs((tskyfi.reshape(lmax+1,-1)-tunlfi)[resttl:resttu,restpl:restpu]-((tskyducc.reshape(lmax+1,-1)-tunlducc.reshape(lmax+1,-1)))[resttl:resttu,restpl:restpu]))**2))
    

In [ ]:
duration = [10,35]
machine = {'mesabi': 24, "mangi": 128, "agare64":64, "agare128":128}
def f(dur, itermax, nsim, nset, ncore):
    return 1/60 * dur*itermax*nsim*nset*8/(ncore)

plt.plot(np.arange(*duration,5), [f(n, 10, 10, 1*3*3, machine['mesabi']) for n in np.arange(*duration,5)], label= 'mesabi')
plt.plot(np.arange(*duration,5), [f(n, 10, 10, 1*3*3, machine['mangi']) for n in np.arange(*duration,5)], label= 'mangi')
plt.plot(np.arange(*duration,5), [f(n, 10, 10, 1*3*3, machine['agare64']) for n in np.arange(*duration,5)], label= 'agare64')
plt.plot(np.arange(*duration,5), [f(n, 10, 10, 1*3*3, machine['agare128']) for n in np.arange(*duration,5)], label= 'agare128')

plt.plot(np.arange(*duration,5), [f(n, 10, 100, 5*3, 24) for n in np.arange(*duration,5)], label='CSCS', ls='--', lw=1)
plt.plot(np.arange(*duration,5), [f(n, 10, 100, 5*2, 256) for n in np.arange(*duration,5)], label='NERSC', ls='--', lw=1)
plt.legend()
plt.yscale('log')
plt.xlabel("minutes per iteration per 8 CPUs")
plt.title("1 mask, 5 foregrounds, 3 estimators, 100 sims")
plt.ylabel("node hours")